# ConversationKGMemory

지식 그래프의 힘을 활용하여 정보를 저장하고 불러옵니다.

이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는 데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킵니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


In [3]:
# Bedrock 설정 및 Util 함수 설정
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [4]:
from langchain.memory import ConversationKGMemory

In [5]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."},
)

In [6]:
memory.load_memory_variables({"input": "김셜리씨는 누구입니까?"})

{'history': []}

## Chain 에 메모리 활용하기

`ConversationChain` 에 `ConversationKGMemory` 를 메모리로 지정하여 대화를 나눈 후 memory 를 확인해 보도록 하겠습니다.


In [7]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

C:\Users\younghl\AppData\Local\Temp\ipykernel_18592\2545022904.py:19: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation_with_kg = ConversationChain(


첫 번째 대화를 시작합니다. 간단한 인물에 대한 정보를 제공해 보겠습니다.


In [8]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
)

"Hi Teddy! Nice to meet you. It sounds like you have a new team member - that's exciting! Having Shirley join as a new designer must bring some fresh perspectives to your company. How has it been working with her so far? I'd be happy to chat about work dynamics or anything else you'd like to discuss, though I should mention that I can only work with information you provide me since I don't have any additional context about your specific workplace or colleagues."

Shirley 라는 사람에 대한 질문을 진행합니다.


In [9]:
# Shirley 에 대한 질문
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

{'history': "On Shirley: Shirley is a coworker of Teddy. Shirley is a new designer. Shirley works at Teddy's company."}